In [10]:
import matplotlib.pyplot as plt
import pygame
import numpy as np
import math

In [11]:
##### CONSTANTES

f = 0.3
rb = 1
rt = 2
dt = 0.0
m = 1
g = 9.81

In [12]:
###### FONCTIONS UTILES

def produitvectoriel(vect1,vect2):
    return (vect1[0]*vect2[1] - vect1[1]*vect2[0])

def produitscalaire(vect1,vect2):
    return(vect1[0]*vect2[0] + vect1[1]*vect2[1])




In [13]:
#### CLASSE SEGMENT

class Segment:
    def __init__(self,array1,array2):
        self.depart = array1
        self.arrivee = array2
        
    def cross(self , segment):
        ab = [self.arrivee[0] - self.depart[0] , self.arrivee[1] - self.depart[1]]
        apbp = [segment.arrivee[0] - segment.depart[0] , segment.arrivee[1] - segment.depart[1]]
        abp = [segment.arrivee[0] - self.depart[0], segment.arrivee[1] - self.depart[1]]
        aap = [segment.depart[0] - self.depart[0], segment.depart[1] - self.depart[1] ]
        apb = [self.arrivee[0] - segment.depart[0] , self.arrivee[1] - segment.depart[1]]
        apa = [self.depart[0] - segment.depart[0] , self.depart[1] - segment.depart[1]]
        
        c1 = produitvectoriel(ab,apbp) != 0
        c2 = (produitvectoriel(ab,abp) * produitvectoriel(ab,aap))<=0
        c3 = (produitvectoriel(apbp,apb) * produitvectoriel(apbp,apa))<=0
        return(c1 and c2 and c3)
        

In [14]:

##### CLASSE TROU

class Trou:
    def __init__(self, array):
        self.position = array
 


In [15]:
##### CLASSE BALLE

class Balle:
    def __init__(self, array):
        self.position = array 


In [16]:
##### CLASSE PARTIE

class Partie:
    def __init__(self, arrayballe,arraytrou, obstacles):
        self.balle = Balle(arrayballe)
        self.trou = Trou(arraytrou)
        self.obstacles = obstacles
        self.is_over = False
        self.nb_coups = 0
        self.is_win = False 
        
    def is_won(self):
        distance = np.sqrt((self.trou.position[0] - self.balle.position[0])**2 + (self.trou.position[1] - self.balle.position[1])**2 )
       # print(distance)
        return(distance < rt)
    
    def is_possible_move(self,segment_dep):
        for i in self.obstacles: # Pour tous les obstacles
            if i.cross(segment_dep)== False: # Si un segment_dep et i se croisent alors le move n'est pas possible
                    return True
        return False
    
    
    def deplacement(self,vitesse_initiale): # La vitesse sera de la forme [ ||v|| , theta)]
        distance =0.5*m/f*vitesse_initiale[0]**2
       # print(vitesse_initiale[0])
        segment_dep = Segment([self.balle.position[0],self.balle.position[1]],[self.balle.position[0] + np.cos(vitesse_initiale[1]) * distance,self.balle.position[1] + np.sin(vitesse_initiale[1]) * distance] )
       # print(self.is_possible_move(segment_dep))
        if self.is_possible_move(segment_dep):
            self.balle.position[0] += np.cos(vitesse_initiale[1]) * distance
            self.balle.position[1] += np.sin(vitesse_initiale[1]) * distance  
        else :
            self.is_over = True
        if self.is_won() :
            self.is_win = True
        return (self.balle.position[0],self.balle.position[1])





In [17]:
obstacles = []
# On ajoute les murs
obstacles.append(Segment([0,0],[0,700]))
obstacles.append(Segment([0,0],[700,0]))
obstacles.append(Segment([700,0],[700,700]))
obstacles.append(Segment([0,700],[700,700]))
# On ajoute les obstacles
obstacles.append(Segment([300,300],[900,300]))


array_balle = [500,500]
array_trou = [100,100]
vitesse = [0.3,np.pi/2]
partie = Partie(array_balle,array_trou, obstacles)
print(partie.trou.position[0])
for i in range(10) :
    partie.deplacement(vitesse)
   
pygame.init()
win = pygame.display.set_mode((700,700))
pygame.display.set_caption("Le mini-golf de l'enfer")
vitesse = [4.,3,5,4,6]
angle = [30,180,270,90,160]
x_trou = partie.trou.position[0]
y_trou = partie.trou.position[1]
win.fill((0,128,0))
pygame.draw.line(win, (0,0,0), obstacles[4].depart, obstacles[4].arrivee)
pygame.draw.circle(win, (250,250,250), (int(partie.balle.position[0]),int(partie.balle.position[1])), 10) 
pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), 20) 
pygame.display.update()
run = True
while run :
    pygame.time.delay(30)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
            
    keys = pygame.key.get_pressed()
  
    win.fill((0,128,0))
    pygame.draw.line(win, (0,0,0), obstacles[4].depart, obstacles[4].arrivee)
    pygame.draw.circle(win, (0,0,0), (x_trou,y_trou), 20) 
    pygame.draw.circle(win, (250,250,250), (int(partie.balle.position[0]),int(partie.balle.position[1])), 10)
    pygame.display.update()
    
    
    if keys[pygame.K_SPACE] :
        for vit,ang in zip(vitesse,angle) :
            while vit > 0 :
                pygame.time.delay(30)
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        run = False
                pygame.draw.circle(win, (0,0,0), (100,100), 20) 
                pygame.draw.circle(win, (250,250,250), (int(partie.deplacement([vit,ang])[0]),int(partie.deplacement([vit,ang])[1])), 10)
                pygame.display.update()
                vit-=0.5
                win.fill((0,128,0))
    vel = 10


print("fin")
pygame.display.flip()
pygame.quit()

100
fin
